# Lab ACtivity 7 - Artificial Neural Network

TP noté, à rendre.

## Imports

In [36]:
# Pandas
import pandas as pd
# Sklearn
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
# Numpy
import numpy as np
# Matplotlib
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns
# pip install tensorflow
# Télécharger le code source de TensorFlow pour l'utiliser de façon optimale.
# Tensorflow
import tensorflow as tf
# Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

## 1. Charger les données

In [6]:
# Import des données, avec comme séparateur "," et la colonne d'Index est "CustomId".
df=pd.read_csv('bank_customers.csv', sep=',').set_index('CustomerId')
# Afficher les colonnes et leur type.
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 15634602 to 15628319
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  int64  
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1.1+ MB


## 2. Inspecter et nettoyer les données

In [7]:
# Visualisation des 5 prmières lignes
print(df.head())

# Combien de valeur manquante par colonne ?
print(df.isnull().sum())

            RowNumber   Surname  CreditScore Geography  Gender  Age  Tenure  \
CustomerId                                                                    
15634602            1  Hargrave          619    France  Female   42       2   
15647311            2      Hill          608     Spain  Female   41       1   
15619304            3      Onio          502    France  Female   42       8   
15701354            4      Boni          699    France  Female   39       1   
15737888            5  Mitchell          850     Spain  Female   43       2   

              Balance  NumOfProducts  HasCrCard  IsActiveMember  \
CustomerId                                                        
15634602         0.00              1          1               1   
15647311     83807.86              1          0               1   
15619304    159660.80              3          1               0   
15701354         0.00              2          0               0   
15737888    125510.82              1        

In [8]:
# Suppression des colonnes inutiles, RowNumber et Surname (qui est un string) :
df.drop(['RowNumber', 'Surname'], axis=1, inplace=True)

# Gestion des NaN (aucun NaN)
# df.dropna(inplace=True)

## 3. Encoder de façon catégoriel

In [9]:
# One-hot encoding
colonnes_categorielles = ['Gender', 'Geography']
df = pd.get_dummies(df, columns=colonnes_categorielles)

# Convertir les colonnes résultantes du codage one-hot en int
for col in df.columns:
    if col.startswith(tuple(colonnes_categorielles)):
        df[col] = df[col].astype(int)

## 4. Diviser les données en entrainement et test

In [10]:
# X pour les valeurs générales, on prend tout sauf la colonne résultat.
# y pour le résultat attendu, on prend seulement la colonne résultat.
X = df.drop('Exited', axis=1)  
y = df['Exited']

# Division, à 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 5. StandarScaler pour standardiser

In [11]:
# Standardisation sur l'entrainement et le test. 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 6. Utiliser keras.models pour initialiser ANN en faisant par couche

In [18]:
# Initialiser le ANN
classifier = Sequential()

## 7. Ajouter la couche d'entrée et la première couche cachée en utilidant la méthode add()

In [21]:
# Ajouter la couche d'entrée et la première couche cachée
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=X_train.shape[1]))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## 8. Ajouter une deuxième couche cachée composée de 6 unités

In [22]:
# Ajouter une deuxième couche cachée
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

## 9. Ajouter la couche de sortie avec une unité binaire, et ne pas oublier l'activation sigmoïde fonction à utiliser

In [23]:
# Ajouter la couche de sortie, avec sigmoid.
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

## 10. Configurer le modèle d'apprentissage, via la méthode de comilation

In [24]:
# Compiler le ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## 11. Ajouster le modèle et appliquer la validation croisée avec une taille de lot de 10. Répéter 50 ou 100 fois (époque = 50 ou 100)

In [26]:
# Ajuster le ANN à l'ensemble d'entraînement
classifier.fit(X_train, y_train, batch_size=10, epochs=50)

Train on 8000 samples
Epoch 1/50
8000/8000 [==============================] - 1s 89us/sample - loss: 0.3905 - acc: 0.8415
Epoch 2/50
8000/8000 [==============================] - 1s 86us/sample - loss: 0.3899 - acc: 0.8425
Epoch 3/50
8000/8000 [==============================] - 1s 77us/sample - loss: 0.3864 - acc: 0.8418
Epoch 4/50
8000/8000 [==============================] - 1s 68us/sample - loss: 0.3809 - acc: 0.8431
Epoch 5/50
8000/8000 [==============================] - 1s 68us/sample - loss: 0.3727 - acc: 0.8444
Epoch 6/50
8000/8000 [==============================] - 1s 69us/sample - loss: 0.3620 - acc: 0.8549
Epoch 7/50
8000/8000 [==============================] - 1s 76us/sample - loss: 0.3523 - acc: 0.8611
Epoch 8/50
8000/8000 [==============================] - 1s 77us/sample - loss: 0.3472 - acc: 0.8609
Epoch 9/50
8000/8000 [==============================] - 1s 77us/sample - loss: 0.3452 - acc: 0.8619
Epoch 10/50
8000/8000 [==============================] - 1s 76us/sample - loss

## 12. Faire des prédictions

In [27]:
# Prédire les résultats sur l'ensemble de test
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)


C:\Users\masqu\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


## 13. Evaluer le modèle à l'aide de la matrice de confusion et du rapport de classification

In [28]:
# Matrice de confusion et rapport de classification
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

[[1541   66]
 [ 216  177]]
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.73      0.45      0.56       393

    accuracy                           0.86      2000
   macro avg       0.80      0.70      0.74      2000
weighted avg       0.85      0.86      0.85      2000



## 14. Prédire si le client avec les informations suivantes quittera la banque ou non : Géographie : France • Pointage de crédit : 600 • Sexe : Homme • Âge : 40 • Ancienneté : 3 • Solde : 60000 • Nombre de produits : 2 • Possède un crédit carte : Oui • Est membre actif : Oui • Salaire estimé : 50000

In [34]:
# Exemple de données client
new_customer = [[600, 40, 3, 60000, 2, 1, 1, 50000, 0, 1, 1, 0, 0]]  
new_customer = scaler.transform(new_customer)
new_prediction = classifier.predict(new_customer)
new_prediction = (new_prediction > 0.5)
print(new_prediction)  # True pour quitter, False pour rester


[[False]]


C:\Users\masqu\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## 15. Utiliser des modèles Keras séquentiels dans le cadre du flux de trvail Scikit-Learn via le wrappers trouvés dans la bibliothèque Keras. Suivre les instruction données dans le notebook Python pour comprendre comment utiliser KerasClassifier à partir de keras.wrappers.scikit_learn.

In [37]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=X_train.shape[1]))
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn=build_classifier, batch_size=10, epochs=100)
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10, n_jobs=-1)
mean_accuracy = accuracies.mean()
variance = accuracies.std()


In [38]:
print(f"Précision moyenne: {mean_accuracy*100:.2f}%")
print(f"Variance: {variance:.2f}")


Précision moyenne: 83.81%
Variance: 0.02
